In [2]:
import tarfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import gzip
import Preprocessing as pproc 

directory = os.getcwd()
data_path = os.path.join(directory, 'Data')

# Definining the path for the files
RB_path = os.path.join(data_path, 'RateBeer')
BA_path = os.path.join(data_path, 'BeerAdvocate')
MB_path = os.path.join(data_path, 'matched_beer_data')

In [3]:
#Transformation of the .txt files
pproc.text_to_csv(RB_path, 'ratings')
pproc.text_to_csv(RB_path, 'reviews')

pproc.text_to_csv(BA_path, 'ratings')
pproc.text_to_csv(BA_path, 'reviews')

pproc.text_to_csv(MB_path, 'ratings_ba')
pproc.text_to_csv(MB_path, 'ratings_with_text_ba')
pproc.text_to_csv(MB_path, 'ratings_rb')
pproc.text_to_csv(MB_path, 'ratings_with_text_rb')

In [4]:
#MATCHING BEERS:
MB_beers = pd.read_csv(MB_path + '/beers.csv', header=1)
MB_breweries = pd.read_csv(MB_path + '/breweries.csv', header=1)
MB_users = pd.read_csv(MB_path + '/users.csv', header=1)
MB_users_approx = pd.read_csv(MB_path + '/users_approx.csv', header=1)
MB_ratings = pd.read_csv(MB_path + '/ratings.csv', header=1)


In [4]:
#RATEBEERS WEB:
RB_beers = pd.read_csv(RB_path + '/beers.csv')
RB_breweries = pd.read_csv(RB_path + '/breweries.csv')
RB_users = pd.read_csv(RB_path + '/users.csv')

In [15]:
#BEERADVOCATES WEB:
BA_beers = pd.read_csv(BA_path + '/beers.csv')
BA_breweries = pd.read_csv(BA_path + '/breweries.csv')
BA_users = pd.read_csv(BA_path + '/users.csv')
BA_ratings = pd.read_csv(BA_path + '/ratings.tsv', sep='\t')
BA_reviews = pd.read_csv(BA_path + '/reviews.tsv', sep='\t')

In [5]:
MB_ratings['date'] =pd.to_datetime(MB_ratings.date, unit='s')
MB_ratings['year'] = MB_ratings['date'].dt.year
columns_to_keep = ['user_id', 'rating','year', 'beer_name','user_id.1']
filter_MB_ratings = MB_ratings.filter(columns_to_keep)


In [6]:
filter_MB_ratings.shape

(21964, 5)

So with the previous datasets we have the geographical location and the brewery name. Furthermore, it can be merged by the userID that coincides in both columns.

In [7]:
geo_beers = pd.merge(filter_MB_ratings, MB_users[['user_id','location']], on='user_id', how='inner')

In this way we populate the ratings dataset with the location of each user which provides us a relation between a place and parameters related to the beer.

In [8]:
geo_beers

,user_id,rating,year,beer_name,user_id.1,location
0,erzengel.248045,4.80,2011,Trappistes Rochefort 10,83106,Germany
1,visionthing.639993,3.00,2011,Myanmar Lager Beer,91324,Estonia
2,tiong.608427,3.85,2012,Cantillon Tyrnilambic Baie D’Argousier Lambic,98624,Finland
3,tiong.608427,3.68,2012,Cantillon Pikkulinnun Viskilambic,98624,Finland
4,tiong.608427,4.00,2012,Drie Fonteinen Oude Geuze - Armand & Tommy,98624,Finland
...,...,...,...,...,...,...
21959,thebeerwatcher.630107,3.91,2011,Valeir Divers,137922,Belgium
21960,jonj.982265,4.08,2017,Fantôme Saison,394232,Faroe Islands
21961,maxilouis.784601,3.00,2014,Jupiler,304802,Belgium
21962,simoen.728327,2.88,2014,Redenaar,276666,Belgium


In [9]:
geo_beers.location.nunique()
geo_beers.user_id.nunique()

879

In [10]:
location_count = geo_beers.groupby('user_id.1', as_index=False).location.nunique()
nationality_check = location_count.sort_values('location', ascending=False)
nationality_check

,user_id.1,location
878,480444,1
0,46,1
1,154,1
2,180,1
3,181,1
...,...,...
18,5332,1
19,5348,1
20,5448,1
21,5464,1


In [11]:
count_locations = geo_beers.groupby('location', as_index=False).user_id.count()

In [12]:
popular_locations = count_locations.sort_values(by='user_id', ascending=False)
most_popular_locations = popular_locations[popular_locations.user_id >= 50]

In [13]:
locat_list = most_popular_locations['location'].tolist()

In [14]:
filtered_geo_beers = geo_beers[geo_beers['location'].isin(locat_list)]

In [15]:
filtered_geo_beers

,user_id,rating,year,beer_name,user_id.1,location
0,erzengel.248045,4.80,2011,Trappistes Rochefort 10,83106,Germany
7,leighton.343447,4.35,2015,Lindemans Oude Gueuze Cuvée René Special Blend...,74136,England
8,leighton.343447,4.75,2013,Ann,74136,England
9,leighton.343447,4.00,2016,Mornin' Delight,74136,England
10,leighton.343447,4.75,2016,Beavertown / Boneyard Bloody Notorious,74136,England
...,...,...,...,...,...,...
21952,buttermilk.478054,4.08,2015,Fierce,402417,"United States, Illinois"
21953,buttermilk.478054,4.99,2016,Juice Pants,402417,"United States, Illinois"
21954,snowride1080.681709,4.25,2014,Dark 'n Curvy,189057,"United States, Illinois"
21957,idler.183543,2.58,2008,Vlasbier,66892,England


Once we have this we might be able to check what are the most popular place in terms of whatever parameter regarding the beer

In [28]:
def unify_location(loc):
    return 'United States' if loc.startswith('United States') else loc

In [30]:
filtered_geo_beers['Location'] = filtered_geo_beers.location.apply(unify_location)

/tmp/ipykernel_606/422876615.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_geo_beers['Location'] = filtered_geo_beers.location.apply(unify_location)


In [31]:
filtered_geo_beers

,user_id,rating,year,beer_name,user_id.1,location,Location
0,erzengel.248045,4.80,2011,Trappistes Rochefort 10,83106,Germany,Germany
7,leighton.343447,4.35,2015,Lindemans Oude Gueuze Cuvée René Special Blend...,74136,England,England
8,leighton.343447,4.75,2013,Ann,74136,England,England
9,leighton.343447,4.00,2016,Mornin' Delight,74136,England,England
10,leighton.343447,4.75,2016,Beavertown / Boneyard Bloody Notorious,74136,England,England
...,...,...,...,...,...,...,...
21952,buttermilk.478054,4.08,2015,Fierce,402417,"United States, Illinois",United States
21953,buttermilk.478054,4.99,2016,Juice Pants,402417,"United States, Illinois",United States
21954,snowride1080.681709,4.25,2014,Dark 'n Curvy,189057,"United States, Illinois",United States
21957,idler.183543,2.58,2008,Vlasbier,66892,England,England
